In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
path="/content/drive/MyDrive/Colab Notebooks/"

Mounted at /content/drive


In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
from datetime import datetime

def convert_time_format(time_str):
    try:
        # Chuyển từ định dạng 2025-02-16T14:50:00 sang datetime object
        time_obj = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S")
        # Chuyển đổi thành định dạng mới: 16/02/2025 14:50:00
        return time_obj.strftime("%d/%m/%Y %H:%M:%S")
    except ValueError:
        return time_str  # Nếu không thể chuyển đổi, trả về giá trị gốc

def crawl_cafef_news(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to retrieve the page")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []

    for item in soup.find_all('div', class_='tlitem'):  # Adjust the class if necessary
        title_tag = item.find('a')
        time_tag = item.find('span', class_='time')  # Adjust the class if necessary

        if title_tag and time_tag:
            title = title_tag.text.strip()
            time = time_tag.text.strip()
            # Chuyển đổi thời gian trước khi thêm vào danh sách
            time = convert_time_format(time)
            articles.append({"title": title, "time": time})

    return articles

def crawl_multiple_pages(n):
    all_news = []
    base_url = "https://cafef.vn/timelinelist/18831/{}.chn"

    for i in range(1, n + 1):
        url = base_url.format(i)
        news_data = crawl_cafef_news(url)
        print(f"Page {i}: Collected {len(news_data)} articles")
        all_news.extend(news_data)

    return all_news

def save_to_csv(news_data, filename=f"{path}news_data.csv"):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["title", "time"])
        writer.writeheader()
        writer.writerows(news_data)

n = 2400  # Change this number as needed
news_data = crawl_multiple_pages(n)
save_to_csv(news_data)

print(f"Total articles collected: {len(news_data)}")


Page 1: Collected 15 articles
Page 2: Collected 15 articles
Page 3: Collected 15 articles
Page 4: Collected 15 articles
Page 5: Collected 15 articles
Page 6: Collected 15 articles
Page 7: Collected 15 articles
Page 8: Collected 15 articles
Page 9: Collected 15 articles
Page 10: Collected 15 articles
Page 11: Collected 15 articles
Page 12: Collected 15 articles
Page 13: Collected 15 articles
Page 14: Collected 15 articles
Page 15: Collected 15 articles
Page 16: Collected 15 articles
Page 17: Collected 15 articles
Page 18: Collected 15 articles
Page 19: Collected 15 articles
Page 20: Collected 15 articles
Page 21: Collected 15 articles
Page 22: Collected 15 articles
Page 23: Collected 15 articles
Page 24: Collected 15 articles
Page 25: Collected 15 articles
Page 26: Collected 15 articles
Page 27: Collected 15 articles
Page 28: Collected 15 articles
Page 29: Collected 15 articles
Page 30: Collected 15 articles
Page 31: Collected 15 articles
Page 32: Collected 15 articles
Page 33: Collecte

In [ ]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=77bc25d4e2273424177305955760ff1aaae1f3892bd94ec9675655c80d47fa6f
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
import feedparser
import csv
from datetime import datetime

# Hàm chuyển đổi định dạng thời gian từ RSS feed (published)
def convert_time_format(time_str):
    try:
        # Chuyển từ định dạng thời gian của RSS (ví dụ: Sun, 16 Feb 25 18:57:00 +0700)
        time_obj = datetime.strptime(time_str, "%a, %d %b %y %H:%M:%S %z")
        # Chuyển đổi thành định dạng mới: 16/02/2025 14:50:00
        return time_obj.strftime("%d/%m/%Y %H:%M:%S")
    except ValueError:
        return time_str  # Nếu không thể chuyển đổi, trả về giá trị gốc

def crawl_rss_feed(url):
    # Phân tích RSS feed
    feed = feedparser.parse(url)

    articles = []

    # Duyệt qua tất cả các bài viết trong RSS feed
    for entry in feed.entries:
        title = entry.title
        published = entry.published  # Thời gian đăng

        # Chuyển đổi thời gian về định dạng mong muốn
        published = convert_time_format(published)

        articles.append({"title": title, "time": published})

    return articles

def save_to_csv(news_data, filename=f"{path}rss_news_data.csv"):
    # Lưu các bài viết vào file CSV
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["title", "time"])  # Chuyển "published" thành "time"
        writer.writeheader()
        writer.writerows(news_data)

# URL RSS feed
rss_url = "https://cafef.vn/thi-truong-chung-khoan.rss"
news_data = crawl_rss_feed(rss_url)

# Lưu kết quả vào CSV
save_to_csv(news_data)

# In ra số bài báo đã thu thập
print(f"Total articles collected: {len(news_data)}")


Total articles collected: 500


In [ ]:
import pandas as pd

# Đọc dữ liệu từ hai file CSV
df1 = pd.read_csv(f"{path}news_data.csv")
df2 = pd.read_csv(f"{path}rss_news_data.csv")

# Gộp hai DataFrame lại (không loại bỏ trùng lặp)
df_combined = pd.concat([df1, df2], ignore_index=True)

# Chuyển cột 'time' từ kiểu chuỗi thành datetime để dễ dàng sắp xếp
df_combined["time"] = pd.to_datetime(df_combined["time"], format="%d/%m/%Y %H:%M:%S")

# Sắp xếp theo cột 'time' từ gần nhất đến xa nhất
df_combined = df_combined.sort_values(by="time", ascending=False)

# Lưu kết quả vào file CSV mới
df_combined.to_csv(f"{path}merged_cafef_data.csv", index=False, encoding='utf-8')

# In ra số lượng bài báo đã gộp
print(f"Total articles collected: {len(df_combined)}")


Total articles collected: 36500


In [ ]:
import pandas as pd

# Đọc dữ liệu từ tệp CSV
data = pd.read_csv(f"{path}merged_cafef_datacsv")

# Đảm bảo rằng cột 'time' là kiểu datetime
data['time'] = pd.to_datetime(data['time'], errors='coerce')

# Lọc dữ liệu theo khoảng thời gian từ 30/09/2019 đến 30/09/2024
start_date = '2019-12-31'
end_date = '2024-12-31'
filtered_data = data[(data['time'] >= start_date) & (data['time'] <= end_date)]

# Chuyển đổi định dạng thời gian thành dd-mm-yyyy (không có giờ)
filtered_data['time'] = filtered_data['time'].dt.strftime('%d-%m-%Y')

# Lưu lại kết quả đã lọc vào một tệp CSV mới (nếu cần)
filtered_data.to_csv(f'{path}filtered_data.csv', index=False)

# Hiển thị 5 dòng đầu của dữ liệu đã được lọc và chuyển đổi
print(filtered_data.head())

                                                  title        time
1565  Chứng khoán ngày mai, 31-12: Tiền có thể dồn v...  30-12-2024
1566  Góc nhìn CTCK: Tiếp tục rung lắc chờ thông tin...  30-12-2024
1567  Vợ một nhân sự cấp cao của Hoàng Anh Gia Lai m...  30-12-2024
1568  Vietnam Airlines ấn định ngày họp bất thường s...  30-12-2024
1569     Một cổ phiếu ngân hàng ngược dòng tăng lịch sử  30-12-2024


<ipython-input-16-d803ac30cb37>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['time'] = filtered_data['time'].dt.strftime('%d-%m-%Y')


In [ ]:
import pandas as pd

# Đọc dữ liệu đã lọc từ tệp CSV (cập nhật đường dẫn nếu cần)
filtered_data = pd.read_csv(f'{path}filtered_data.csv')

# Đảm bảo rằng cột 'time' là kiểu datetime
filtered_data['time'] = pd.to_datetime(filtered_data['time'], format='%d-%m-%Y')

# Sắp xếp dữ liệu theo ngày
filtered_data = filtered_data.sort_values(by='time')

# Tạo danh sách các ngày liên tiếp từ ngày đầu đến ngày cuối
date_range = pd.date_range(start=filtered_data['time'].min(), end=filtered_data['time'].max(), freq='D')

# Kiểm tra xem tất cả các ngày có trong dữ liệu không
missing_dates = set(date_range) - set(filtered_data['time'])

# Kiểm tra có ngày bị thiếu không và in kết quả
if len(missing_dates) == 0:
    print("Tất cả các ngày trong khoảng thời gian đều có mặt và liên tục.")
else:
    print(f"Có {len(missing_dates)} ngày bị thiếu trong dữ liệu.")
    print("Các ngày thiếu:", sorted(list(missing_dates)))

Tất cả các ngày trong khoảng thời gian đều có mặt và liên tục.
